In [212]:
from app.config import MODEL_NAME, GOOGLE_APPLICATION_CREDENTIALS, GOOGLE_PROJECT_NAME
from app.dependencies import datastore, executor, loop, chat

from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import os


In [213]:
print(MODEL_NAME, GOOGLE_APPLICATION_CREDENTIALS, GOOGLE_PROJECT_NAME)

gemini-1.5-pro C:\Users\T_B\AppData\Roaming\gcloud\application_default_credentials.json great-1165


In [274]:
embeddings = VertexAIEmbeddings(model="text-embedding-004", project=os.getenv("GOOGLE_PROJECT_NAME"))
model = VertexAI(model="gemini-1.5-pro", temperature=0.2)


In [275]:
text_splitter = RecursiveCharacterTextSplitter()

loader = DirectoryLoader('data', glob="**/static/*.txt")
docs = loader.load()

documents = text_splitter.split_documents(docs)

vector = FAISS.from_documents(documents, embeddings)
vector.save_local("../data/provisoire/vector_store.faiss")

retriever = vector.as_retriever()

In [276]:
precision = """
    Tu es Dieter Hoffman et tu dois choisir ta prochaine action.

    Maintenant, ton action est : Stop, None.

    Autour de toi, tu as Farida Wang et Laurent Dubois. 
    Tu as entendu Laurent Dubois dire : "Emma devrait revoir ses cours d'allemand."
    
    Tu as le choix entre 4 actions que tu peux réaliser : 
        - Speak : La prochaine action est de parler.
        - Move : La prochaine action est de vous déplacer à quelqu'un. 
        - Stop : La prochaine action est de s'arrêter.
        - ContinueAction : Tu continues l'action que tu as actuellement.
    
    
    *INSTRUCTIONS:
    Rules:
        - Your role is to choose your next action
        - Choose between Speak, Move and ContinueAction.
        - If you choose Speak or Move, you have to point someone.
        - Hearing someone talk doesn't necessarily merit a response.
        - Hearing someone talk doesn't necessarily merit to move to her.
        - If you are stop, you need to choose between Speak, Move and ContinueAction.
    Output response format:
        - Only one of the 4 actions can be chosen.
        - If you choose Speak, you have to point someone. return: Move, name
        - If you choose Move, you have to point someone. return: Speak, name
        - If you choose Stop. return: Stop, None
        - If you choose ContinueAction. return: ContinueAction, None


"""

In [283]:

# Define prompt template
prompt = ChatPromptTemplate.from_template("""
    <context> {context} </context> \n
    Precisions :  {input} \n
    Question : What action do I need to do now?"""
    )

# Create a retrieval chain to answer questions
# The create_stuff_documents_chain function combines a language model and a prompt template
# to generate a response based on a set of documents.
document_chain = create_stuff_documents_chain(model, prompt)

# Create a retrieval-augmented chain
# The create_retrieval_chain function combines a retriever and a document chain to generate
# a response based on a set of documents retrieved from the retriever.
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Invoke the retrieval-augmented chain
# The invoke method of the retrieval_chain is used to generate a response based on the input
# parameters. In this case, the input parameters are the user's question and the prompt for the
# language model.
response = retrieval_chain.invoke({"input": precision})



Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.c

In [284]:
# Return the answer from the response
response["answer"]

'ContinueAction, None \n'